# Nuclei segmentation

This notebook is done as an approachable wy of assessing the nuclei segmentation with napari and rerun the calculations.

In [1]:
from ipywidgets import interact, interactive
from IPython.display import display
import pandas as pd
import pathlib

from nuclei_segmenter import loader as io, vis, quantify as quant

In [2]:
DATA_DIR = pathlib.Path(r'P:\c5_Andersson_Emma\Afshan\Nuclei Area quantication')
IMPR_DIR = pathlib.Path(r'P:\c5_Andersson_Emma\Afshan\Nuclei Area quantication\segmented_nuclei\segmented')
file_list = [filepath for filepath in DATA_DIR.iterdir() if filepath.is_file()
             and filepath.suffix == '.czi'
             and 'Orthogonal' not in filepath.name
             and 'Subset' not in filepath.name]

file_dict = {this_file.stem: {'img_path': this_file, 'labeled_path': IMPR_DIR / (this_file.stem + '.tiff')} for this_file in file_list}

First you have to choose the file to open.

In [3]:
def choose_file(file):
    return file

def load_imgs(file):
    img = io.get_image(io.get_file(file_dict[file]['img_path']))
    labeled = io.get_labeled(file_dict[file]['labeled_path'])
    scale = io.get_pixel_size(io.get_file(file_dict[file]['img_path']))
    return img, labeled, scale

w = interactive(choose_file, file=list(file_dict.keys()))
display(w)

interactive(children=(Dropdown(description='file', options=('Ndr1219 p8 Hilar-3', 'Ndr1219 p8 Hilar-4', 'Ndr12…

Once you selected the file, run the following cell to see the images.

In [4]:
file = w.result
img, labels, scale = load_imgs(file)

### 2D viewer

In [5]:
vis.view_2d(img, labels)

### 3D Viewer

In [6]:
vis.view_3d(img, labels)

### Area colored

In [19]:
def get_labeled_by_area(img, labeled, scale):
    nuclei_props = quant.get_all_props(img, labeled)
    nuclei_props = quant.calculate_area(nuclei_props, scale)
    nuclei_props = quant.calculate_volume(nuclei_props, scale)

    return vis.relabel_by(labeled, nuclei_props)

In [20]:
area_labeled = get_labeled_by_area(img, labels, scale)

In [21]:
vis.view_property_2d(img, area_labeled)

## Save new labels

If you modified anything from the labeled image and you want to save it, run the cell below.

In [14]:
vis.save_img(file_dict[file]['labeled_path'], labels)

# ReQuantify

After modifying the labels from the segmentation, you can run the following cells to update the excel files with the quantifications.

In [15]:
!python requantify.py

ReAnalyzing Ndr1219 p8 Hilar-3.tiff
Loading image
Quantifying Ndr1219 p8 Hilar-3.tiff
Saving at P:\c5_Andersson_Emma\Afshan\Nuclei Area quantication\segmented_nuclei\segmented\Ndr1219 p8 Hilar-3.xlsx
ReAnalyzing Ndr1219 p8 Hilar-4.tiff
Loading image
Quantifying Ndr1219 p8 Hilar-4.tiff
Saving at P:\c5_Andersson_Emma\Afshan\Nuclei Area quantication\segmented_nuclei\segmented\Ndr1219 p8 Hilar-4.xlsx
ReAnalyzing Ndr1224 Hilar-1.tiff
Loading image
Quantifying Ndr1224 Hilar-1.tiff
Saving at P:\c5_Andersson_Emma\Afshan\Nuclei Area quantication\segmented_nuclei\segmented\Ndr1224 Hilar-1.xlsx
ReAnalyzing Ndr1224 Hilar-2.tiff
Loading image
Quantifying Ndr1224 Hilar-2.tiff
Saving at P:\c5_Andersson_Emma\Afshan\Nuclei Area quantication\segmented_nuclei\segmented\Ndr1224 Hilar-2.xlsx
ReAnalyzing Ndr1224 Hilar-3.tiff
Loading image
Quantifying Ndr1224 Hilar-3.tiff
Saving at P:\c5_Andersson_Emma\Afshan\Nuclei Area quantication\segmented_nuclei\segmented\Ndr1224 Hilar-3.xlsx
ReAnalyzing Ndr1224 Hilar-4


100%|██████████| 11/11 [01:09<00:00,  6.31s/it]


# Generate Report

Once you are happy with the new segmentations and you have requantified the images, you can generate a new report with the following cell.

Structure of this command is python report.py [upper limit of color for image color coded for area] [upper limit of histogram] [bin width of histograms]

Choose the upper limit for the image color coded for area so that the middle point is close to the value that separates small from big nuclei. I chose 400 because: nucleai smaller than 200 will be blue and bigger than 200 will be red.

Choose the upper limit of histogram so that the biggest nuclei is included.

Choose the bin width according to what resolution you want (don't use this to classify nuclei by size).

In [39]:
!python report.py 400 800 25

Preparing Ndr1219 p8 Hilar-3.tiff
Loading image
Relabeling image
Estimating good planes
Saving report at P:\c5_Andersson_Emma\Afshan\Nuclei Area quantication\segmented_nuclei\segmented\Ndr1219 p8 Hilar-3.pdf
Saving big excel file
Preparing Ndr1219 p8 Hilar-4.tiff
Loading image
Relabeling image
Estimating good planes
Saving report at P:\c5_Andersson_Emma\Afshan\Nuclei Area quantication\segmented_nuclei\segmented\Ndr1219 p8 Hilar-4.pdf
Saving big excel file
Preparing Ndr1224 Hilar-1.tiff
Loading image
Relabeling image
Estimating good planes
Saving report at P:\c5_Andersson_Emma\Afshan\Nuclei Area quantication\segmented_nuclei\segmented\Ndr1224 Hilar-1.pdf
Saving big excel file
Preparing Ndr1224 Hilar-2.tiff
Loading image
Relabeling image
Estimating good planes
Saving report at P:\c5_Andersson_Emma\Afshan\Nuclei Area quantication\segmented_nuclei\segmented\Ndr1224 Hilar-2.pdf
Saving big excel file
Preparing Ndr1224 Hilar-3.tiff
Loading image
Relabeling image
Estimating good planes
Saving 


 64%|██████▎   | 7/11 [02:58<00:58, 14.53s/it]C:\Users\agucor\OneDrive - Karolinska Institutet\Dokument\Lab\Organoid\src\report.py:55: UserWarning: Data has no positive values, and therefore cannot be log-scaled.
  pp.savefig()

100%|██████████| 11/11 [04:11<00:00, 22.89s/it]
